In [12]:
#notebook for exploring and preprocessing the NYT bestseller list dataset
# obtained from https://www.kaggle.com/cmenca/new-york-times-hardcover-fiction-best-sellers/version/1
# originally downloaded from NYT
# there is a kernel associated with this dataset, I use some methods from it
# the use of Levenshtein distance for checking integrity is my work

import json
import pandas as pd
import numpy as np

import re

from string import ascii_lowercase
from datetime import datetime

In [25]:
import sys
!{sys.executable} -m pip install python-Levenshtein

    100% |████████████████████████████████| 51kB 4.6MB/s ta 0:00:011
  Running setup.py bdist_wheel for python-Levenshtein ... done
  Stored in directory: /Users/cstoneki/Library/Caches/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [31]:
import Levenshtein as lev

In [2]:
file = 'nyt2.json'
raw_books = pd.read_json(file, lines=True, orient='columns')
raw_books.head()

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list
0,{'$oid': '5b4aa4ead3089013507db18b'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",{'$numberInt': '27'},Bantam,ODD HOURS,{'$numberInt': '1'},{'$numberInt': '0'},{'$numberInt': '1'}
1,{'$oid': '5b4aa4ead3089013507db18c'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,{'$numberDouble': '25.99'},"Little, Brown",THE HOST,{'$numberInt': '2'},{'$numberInt': '1'},{'$numberInt': '3'}
2,{'$oid': '5b4aa4ead3089013507db18d'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,{'$numberDouble': '24.95'},St. Martin's,LOVE THE ONE YOU'RE WITH,{'$numberInt': '3'},{'$numberInt': '2'},{'$numberInt': '2'}
3,{'$oid': '5b4aa4ead3089013507db18e'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,{'$numberDouble': '22.95'},Putnam,THE FRONT,{'$numberInt': '4'},{'$numberInt': '0'},{'$numberInt': '1'}
4,{'$oid': '5b4aa4ead3089013507db18f'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,{'$numberDouble': '24.95'},Doubleday,SNUFF,{'$numberInt': '5'},{'$numberInt': '0'},{'$numberInt': '1'}


In [3]:
#following is from kaggle kernel
# https://www.kaggle.com/tobaotic/clean-data

regex = re.compile('[^a-z0-9]')

#convert string to lowercase and replace all non alphanumeric characters
def set_lc_values(col_name):
    new_values = []
    for values in raw_books[col_name]:
        #this strips away all whitespace and makes everything lowercase
        new_values.append(regex.sub('', values.lower()))

    return new_values



def flatten_json(y):
    # extract values from json string in dataframe
    
    out = {}
    ret_val = '-'
    
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        elif type(x) is str:
            out[name[:-1]] = x


    flatten(y)

    if type(out) is dict:
        for v in out:
            ret_val = out[v]
    
    return ret_val


In [4]:
#also from kernel

for i in range(len(raw_books)):
    raw_books.loc[i]["_id"] = flatten_json(raw_books.loc[i]["_id"])
    raw_books.loc[i]["bestsellers_date"] = datetime.fromtimestamp(int(flatten_json(raw_books.loc[i]["bestsellers_date"])[:10])).strftime('%Y-%m-%d')
    raw_books.loc[i]["price"] = flatten_json(raw_books.loc[i]["price"])
    raw_books.loc[i]["published_date"] = datetime.fromtimestamp(int(flatten_json(raw_books.loc[i]["published_date"])[:10])).strftime('%Y-%m-%d')
    raw_books.loc[i]["rank"] = flatten_json(raw_books.loc[i]["rank"])
    raw_books.loc[i]["rank_last_week"] = flatten_json(raw_books.loc[i]["rank_last_week"])
    raw_books.loc[i]["title"] = flatten_json(raw_books.loc[i]["title"]) 
    raw_books.loc[i]["weeks_on_list"] = flatten_json(raw_books.loc[i]["weeks_on_list"])     
    
raw_books.head()

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list
0,5b4aa4ead3089013507db18b,2008-05-23,2008-06-07,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",27,Bantam,ODD HOURS,1,0,1
1,5b4aa4ead3089013507db18c,2008-05-23,2008-06-07,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,25.99,"Little, Brown",THE HOST,2,1,3
2,5b4aa4ead3089013507db18d,2008-05-23,2008-06-07,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,24.95,St. Martin's,LOVE THE ONE YOU'RE WITH,3,2,2
3,5b4aa4ead3089013507db18e,2008-05-23,2008-06-07,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,22.95,Putnam,THE FRONT,4,0,1
4,5b4aa4ead3089013507db18f,2008-05-23,2008-06-07,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,24.95,Doubleday,SNUFF,5,0,1


In [5]:
# from kaggle kernel
# add three new columns only with alphanumeric values for comparision and spellcheck detection

raw_books['l_author'] = set_lc_values('author')
raw_books['l_publisher'] = set_lc_values('publisher')
raw_books['l_title'] = set_lc_values('title')

raw_books.head()



,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list,l_author,l_publisher,l_title
0,5b4aa4ead3089013507db18b,2008-05-23,2008-06-07,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",27,Bantam,ODD HOURS,1,0,1,deanrkoontz,bantam,oddhours
1,5b4aa4ead3089013507db18c,2008-05-23,2008-06-07,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,25.99,"Little, Brown",THE HOST,2,1,3,stepheniemeyer,littlebrown,thehost
2,5b4aa4ead3089013507db18d,2008-05-23,2008-06-07,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,24.95,St. Martin's,LOVE THE ONE YOU'RE WITH,3,2,2,emilygiffin,stmartins,lovetheoneyourewith
3,5b4aa4ead3089013507db18e,2008-05-23,2008-06-07,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,22.95,Putnam,THE FRONT,4,0,1,patriciacornwell,putnam,thefront
4,5b4aa4ead3089013507db18f,2008-05-23,2008-06-07,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,24.95,Doubleday,SNUFF,5,0,1,chuckpalahniuk,doubleday,snuff


In [14]:
# following is my work
#any given title may appear multiple times
#because it was in multiple weekly bestseller lists
#also, some books may have the same title
# here is an example showing both phenomena:

raw_books.loc[raw_books['title']=="SNUFF",:]


,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list,l_author,l_publisher,l_title
4,5b4aa4ead3089013507db18f,2008-05-23,2008-06-07,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,24.95,Doubleday,SNUFF,5,0,1,chuckpalahniuk,doubleday,snuff
29,5b4aa4ead3089013507db1a8,2008-05-30,2008-06-14,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queen aims to cap her career by ...,24.95,Doubleday,SNUFF,10,5,2,chuckpalahniuk,doubleday,snuff
49,5b4aa4ead3089013507db1bc,2008-06-06,2008-06-21,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queen aims to have sex on film w...,24.95,Doubleday,SNUFF,10,10,3,chuckpalahniuk,doubleday,snuff
77,5b4aa4ead3089013507db1d8,2008-06-13,2008-06-28,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,0,Doubleday,SNUFF,18,0,0,chuckpalahniuk,doubleday,snuff
96,5b4aa4ead3089013507db1eb,2008-06-20,2008-07-05,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,0,Doubleday,SNUFF,17,0,0,chuckpalahniuk,doubleday,snuff
3542,5b4aa4ead3089013507dbf61,2011-10-14,2011-10-29,http://www.amazon.com/Snuff-Discworld-Terry-Pr...,Terry Pratchett,"In this Discworld fantasy, the commander of An...",25.99,Harper/HarperCollins,SNUFF,3,0,1,terrypratchett,harperharpercollins,snuff
3576,5b4aa4ead3089013507dbf83,2011-10-21,2011-11-05,http://www.amazon.com/Snuff-Discworld-Terry-Pr...,Terry Pratchett,"In this Discworld fantasy, the commander of An...",0,Harper/HarperCollins,SNUFF,17,0,0,terrypratchett,harperharpercollins,snuff


In [17]:
#we want to find all unique books and assign each an ID
# bookID = <lc author>_<lc title> 

raw_books['book_ID'] = raw_books['l_author'] + '_' + raw_books['l_title']

In [22]:
print(len(raw_books['book_ID'].unique()))

2328


In [43]:
#check for IDs that are similar
#because these may be spurious duplications from the same book
#that arise due to typo
possible_matches = []
book_ID_groups = raw_books.groupby('book_ID')
for i, (ID1, fields1) in enumerate(book_ID_groups):
    for j, (ID2, fields2) in enumerate(book_ID_groups):
        if(j <= i): continue
        if(lev.distance(ID1, ID2)/max(len(ID1), len(ID2)) < 0.1):
            print("Possible match:")
            print(ID1)
            print(ID2)
            possible_matches.append((ID1, ID2))
    print("Finished checking "+ID1)
print("Finished checking all IDs")

Possible match:
aaronallston_backlash
aaronallston_conviction
Possible match:
aaronallston_backlash
aaronallston_outcast
Finished checking aaronallston_backlash
Possible match:
aaronallston_conviction
aaronallston_outcast
Finished checking aaronallston_conviction
Finished checking aaronallston_outcast
Finished checking aaronallston_starwarsxwingmercykill
Finished checking abrahamverghese_cuttingforstone
Possible match:
aceatkins_robertbparkerscheapshot
aceatkins_robertbparkerskickback
Possible match:
aceatkins_robertbparkerscheapshot
aceatkins_robertbparkerslittlewhitelies
Possible match:
aceatkins_robertbparkerscheapshot
aceatkins_robertbparkerslullaby
Possible match:
aceatkins_robertbparkerscheapshot
aceatkins_robertbparkersslowburn
Possible match:
aceatkins_robertbparkerscheapshot
aceatkins_robertbparkerswonderland
Possible match:
aceatkins_robertbparkerscheapshot
robertknott_robertbparkersbullriver
Possible match:
aceatkins_robertbparkerscheapshot
robertknott_robertbparkersironhors

Finished checking alexandermccallsmith_teatimeforthetraditionallybuilt
Possible match:
alexandermccallsmith_thecomfortsofamuddysaturday
alexandermccallsmith_thedoublecomfortsafariclub
Possible match:
alexandermccallsmith_thecomfortsofamuddysaturday
alexandermccallsmith_theforevergirl
Possible match:
alexandermccallsmith_thecomfortsofamuddysaturday
alexandermccallsmith_thehandsomemansdeluxecaf
Possible match:
alexandermccallsmith_thecomfortsofamuddysaturday
alexandermccallsmith_thehouseofunexpectedsisters
Possible match:
alexandermccallsmith_thecomfortsofamuddysaturday
alexandermccallsmith_thelimpopoacademyofprivatedetection
Possible match:
alexandermccallsmith_thecomfortsofamuddysaturday
alexandermccallsmith_thelostartofgratitude
Possible match:
alexandermccallsmith_thecomfortsofamuddysaturday
alexandermccallsmith_theminoradjustmentbeautysalon
Possible match:
alexandermccallsmith_thecomfortsofamuddysaturday
alexandermccallsmith_thenovelhabitsofhappiness
Possible match:
alexandermccalls

KeyboardInterrupt: 

In [44]:
t = (1,2)
(m,n) = (u + 5 for u in t)

In [46]:
n

7

In [ ]:
#can hand-curate possible matches here...

for (ID1, ID2) in possible_matches:
    (n1, n2) = (np.sum(raw_books['book_ID']==ID) for ID in (ID1, ID2))
    if(n1 > n2): raw_books.loc[raw_books['book_ID']==ID2, 'book_ID'] = ID1
           else: raw_books.loc[raw_books['book_ID']==ID1, 'book_ID'] = ID2

In [ ]:
#now we have hopefully gathered the correct set of entries under the correct book_ID


In [53]:
np.array([3,2]).shape

(2,)

In [56]:
def get_majority_string(ls):
    u = np.unique(ls)
    if(len(u)==1): return u[0]
    else: 
        counts = np.array([np.sum(np.array([s==v for s in ls])) for v in u])
        ind = np.argmax(counts)
        if(len(ind.shape)==0): return u[ind] #if ind is a scalar
        else: return u[ind[0]]

In [64]:
#do integrity checking on publishers
unique_publishers = raw_books['publisher'].unique()
possible_matches = []
for i, p1 in enumerate(unique_publishers):
    for j, p2 in enumerate(unique_publishers):
        if(i <= j): continue
        if(lev.distance(p1, p2)/max(len(p1), len(p2)) < 0.15):
            print("Possible match:")
            print(p1)
            print(p2)
            possible_matches.append((p1, p2))

Possible match:
St. Martin’s
St. Martin's
Possible match:
Viking
Viking,
Possible match:
Double­day
Doubleday
Possible match:
Del Rey/ Ballantine
Del Rey/Ballantine
Possible match:
Ballantine,
Ballantine
Possible match:
allantine
Ballantine
Possible match:
St. Martin''s
St. Martin's
Possible match:
Tor/ Tom Doherty
Tor/Tom Doherty
Possible match:
Tor/TomDoherty
Tor/Tom Doherty
Possible match:
Tor/TomDoherty
Tor/ Tom Doherty
Possible match:
Nan. A Talese/Doubleday
Nan A. Talese/Doubleday
Possible match:
St.Martin’s
St. Martin’s
Possible match:
Little Brown
Little, Brown
Possible match:
Thomas Dunne/St. Martin's
Thomas Dunne/St. Martin’s
Possible match:
St. Martin‘s
St. Martin's
Possible match:
St. Martin‘s
St. Martin’s
Possible match:
Marian Wood/Putnam
Marion Wood/Putnam
Possible match:
Little, Brown & Company
Little Brown & Company
Possible match:
HarperVoyager
Harper Voyager
Possible match:
Nan A. Talese/ Doubleday
Nan A. Talese/Doubleday
Possible match:
Nan A. Talese/ Doubleday
Nan.

In [68]:
("a", "b")==("a", "x")

False

In [ ]:
#replace with majority
#go through publishers in order of size (number of records)
#and take all matches
replacements = []
publisher_counts = np.array([np.sum(raw_books["publisher"]==p) for p in unique_publishers])
order = np.argsort(publisher_counts)
for ind in order:
    p1 = unique_publishers[ind]
    to_remove = []
    for t in possible_matches:
        if(p1 in t):
            other = [u for u in t if u!=p1]
            replacements.append((other, p1))
            to_remove.append((other, p1))
    for t in to_remove:
        possible_matches.remove(t)
#actually perform replacements here
for t in replacements:
    raw_books.loc[raw_books['publisher']==t[0], 'publisher'] = t[1]
    

In [58]:
get_majority_string(['stuff', 'x', 'stuff', 'y', 'x', 'x'])

'x'

In [10]:
print('Total number of titles: %s' % (len(raw_books['title'].unique())))
print('Total number of clean titles: %s' % (len(raw_books['l_title'].unique())))

Total number of titles: 2300
Total number of clean titles: 2299


In [20]:
raw_books.head()

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list,l_author,l_publisher,l_title,book_ID
0,5b4aa4ead3089013507db18b,2008-05-23,2008-06-07,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",27,Bantam,ODD HOURS,1,0,1,deanrkoontz,bantam,oddhours,deanrkoontz_oddhours
1,5b4aa4ead3089013507db18c,2008-05-23,2008-06-07,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,25.99,"Little, Brown",THE HOST,2,1,3,stepheniemeyer,littlebrown,thehost,stepheniemeyer_thehost
2,5b4aa4ead3089013507db18d,2008-05-23,2008-06-07,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,24.95,St. Martin's,LOVE THE ONE YOU'RE WITH,3,2,2,emilygiffin,stmartins,lovetheoneyourewith,emilygiffin_lovetheoneyourewith
3,5b4aa4ead3089013507db18e,2008-05-23,2008-06-07,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,22.95,Putnam,THE FRONT,4,0,1,patriciacornwell,putnam,thefront,patriciacornwell_thefront
4,5b4aa4ead3089013507db18f,2008-05-23,2008-06-07,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,24.95,Doubleday,SNUFF,5,0,1,chuckpalahniuk,doubleday,snuff,chuckpalahniuk_snuff


In [36]:
for gi in g: print(gi)

('AJ Finn',                             _id bestsellers_date published_date  \
9790   5b4aa4ead3089013507dd7c9       2018-01-05     2018-01-20   
9805   5b4aa4ead3089013507dd7d8       2018-01-12     2018-01-27   
9820   5b4aa4ead3089013507dd7e7       2018-01-19     2018-02-03   
9835   5b4aa4ead3089013507dd7f6       2018-01-26     2018-02-10   
9852   5b4aa4ead3089013507dd807       2018-02-02     2018-02-17   
9867   5b4aa4ead3089013507dd816       2018-02-09     2018-02-24   
9882   5b4aa4ead3089013507dd825       2018-02-16     2018-03-03   
9897   5b4aa4ead3089013507dd834       2018-02-23     2018-03-10   
9912   5b4aa4ead3089013507dd843       2018-03-02     2018-03-17   
9929   5b4aa4ead3089013507dd854       2018-03-09     2018-03-24   
9944   5b4aa4ead3089013507dd863       2018-03-16     2018-03-31   
9961   5b4aa4ead3089013507dd874       2018-03-23     2018-04-07   
9976   5b4aa4ead3089013507dd883       2018-03-30     2018-04-14   
9993   5b4aa4ead3089013507dd894       2018-04-06  

('Anne Bishop',                            _id bestsellers_date published_date  \
839   5b4aa4ead3089013507db4d2       2009-03-06     2009-03-21   
1875  5b4aa4ead3089013507db8de       2010-03-06     2010-03-20   
2913  5b4aa4ead3089013507dbcec       2011-03-04     2011-03-19   
7095  5b4aa4ead3089013507dcd42       2015-03-06     2015-03-21   
8159  5b4aa4ead3089013507dd16a       2016-03-11     2016-03-26   
9159  5b4aa4ead3089013507dd552       2017-03-10     2017-03-25   

                                     amazon_product_url       author  \
839   http://www.amazon.com/Shadow-Queen-Black-Jewel...  Anne Bishop   
1875  http://www.amazon.com/Shaladors-Lady-Black-Jew...  Anne Bishop   
2913  http://www.amazon.com/Twilights-Dawn-Black-Jew...  Anne Bishop   
7095  http://www.amazon.com/Vision-Silver-A-Novel-Ot...  Anne Bishop   
8159  http://www.amazon.com/Marked-In-Flesh-Novel-Ot...  Anne Bishop   
9159  https://www.amazon.com/Etched-Bone-Novel-Other...  Anne Bishop   

                

10143             beforethestorm             christiegolden_beforethestorm  )
('Christina Baker Kline',                            _id bestsellers_date published_date  \
9121  5b4aa4ead3089013507dd52c       2017-02-24     2017-03-11   
9137  5b4aa4ead3089013507dd53c       2017-03-03     2017-03-18   
9154  5b4aa4ead3089013507dd54d       2017-03-10     2017-03-25   
9171  5b4aa4ead3089013507dd55e       2017-03-17     2017-04-01   

                                     amazon_product_url  \
9121  https://www.amazon.com/Piece-World-Christina-B...   
9137  https://www.amazon.com/Piece-World-Christina-B...   
9154  https://www.amazon.com/Piece-World-Christina-B...   
9171  https://www.amazon.com/Piece-World-Christina-B...   

                     author  \
9121  Christina Baker Kline   
9137  Christina Baker Kline   
9154  Christina Baker Kline   
9171  Christina Baker Kline   

                                            description price  \
9121  Imagining the life of the central figure o

[69 rows x 16 columns])
('Danielle Steel',                             _id bestsellers_date published_date  \
103    5b4aa4ead3089013507db1f2       2008-06-27     2008-07-12   
127    5b4aa4ead3089013507db20a       2008-07-04     2008-07-19   
152    5b4aa4ead3089013507db223       2008-07-11     2008-07-26   
464    5b4aa4ead3089013507db35b       2008-10-31     2008-11-15   
489    5b4aa4ead3089013507db374       2008-11-07     2008-11-22   
...                         ...              ...            ...   
9972   5b4aa4ead3089013507dd87f       2018-03-30     2018-04-14   
9992   5b4aa4ead3089013507dd893       2018-04-06     2018-04-21   
10075  5b4aa4ead3089013507dd8e6       2018-05-18     2018-06-02   
10091  5b4aa4ead3089013507dd8f6       2018-05-25     2018-06-09   
10113  5b4aa4ead3089013507dd90c       2018-06-01     2018-06-16   

                                      amazon_product_url          author  \
103    http://www.amazon.com/Rogue-Danielle-Steel/dp/...  Danielle Steel   


4193  elizabethlowell_beautifulsacrifice  )
('Elizabeth Peters',                            _id bestsellers_date published_date  \
295   5b4aa4ead3089013507db2b2       2008-08-29     2008-09-13   
1964  5b4aa4ead3089013507db937       2010-04-10     2010-04-24   
1995  5b4aa4ead3089013507db956       2010-04-17     2010-05-01   
2019  5b4aa4ead3089013507db96e       2010-04-24     2010-05-08   

                                     amazon_product_url            author  \
295   http://www.amazon.com/Laughter-Dead-Kings-Vick...  Elizabeth Peters   
1964  http://www.amazon.com/River-Sky-Amelia-Peabody...  Elizabeth Peters   
1995  http://www.amazon.com/River-Sky-Amelia-Peabody...  Elizabeth Peters   
2019  http://www.amazon.com/River-Sky-Amelia-Peabody...  Elizabeth Peters   

                                            description  price  \
295   The art historian and sleuth Vicky Bliss must ...  25.95   
1964  In 1910, the Peabody-Emerson clan tries to hea...  25.99   
1995  In 1910, the P

...

[94 rows x 16 columns])
('Leila Meacham',                            _id bestsellers_date published_date  \
1732  5b4aa4ead3089013507db84f       2010-01-16     2010-01-30   
1748  5b4aa4ead3089013507db85f       2010-01-23     2010-02-06   
1769  5b4aa4ead3089013507db874       2010-01-30     2010-02-13   
1787  5b4aa4ead3089013507db886       2010-02-06     2010-02-20   
1812  5b4aa4ead3089013507db89f       2010-02-13     2010-02-27   
1838  5b4aa4ead3089013507db8b9       2010-02-20     2010-03-06   
5976  5b4aa4ead3089013507dc8e3       2014-02-07     2014-02-22   
5995  5b4aa4ead3089013507dc8f6       2014-02-14     2014-03-01   

                                     amazon_product_url         author  \
1732  http://www.amazon.com/Roses-Leila-Meacham/dp/0...  Leila Meacham   
1748  http://www.amazon.com/Roses-Leila-Meacham/dp/0...  Leila Meacham   
1769  http://www.amazon.com/Roses-Leila-Meacham/dp/0...  Leila Meacham   
1787  http://www.amazon.com/Roses-Leila-Meacham/dp/0...  Leila Meac

10059   riverhead  thefemalepersuasion  megwolitzer_thefemalepersuasion  )
('Megan McCafferty',                           _id bestsellers_date published_date  \
952  5b4aa4ead3089013507db543       2009-04-17     2009-05-02   

                                    amazon_product_url            author  \
952  http://www.amazon.com/Perfect-Fifths-Jessica-D...  Megan McCafferty   

                                       description price publisher  \
952  The final book in the Jessica Darling series.    22     Crown   

              title rank rank_last_week weeks_on_list         l_author  \
952  PERFECT FIFTHS   13              0             1  meganmccafferty   

    l_publisher        l_title                        book_ID  
952       crown  perfectfifths  meganmccafferty_perfectfifths  )
('Melanie Benjamin',                            _id bestsellers_date published_date  \
4878  5b4aa4ead3089013507dc499       2013-01-18     2013-02-02   
4934  5b4aa4ead3089013507dc4d1       2013-02-08 

('Raymond E Feist',                            _id bestsellers_date published_date  \
915   5b4aa4ead3089013507db51e       2009-04-03     2009-04-18   
3999  5b4aa4ead3089013507dc12a       2012-03-16     2012-03-31   
5217  5b4aa4ead3089013507dc5ec       2013-05-17     2013-06-01   

                                     amazon_product_url           author  \
915   http://www.amazon.com/Rides-Dread-Legion-Book-...  Raymond E Feist   
3999  http://www.amazon.com/A-Crown-Imperiled-Chaosw...  Raymond E Feist   
5217  http://www.amazon.com/Magicians-End-Book-Three...  Raymond E Feist   

                                            description  price  \
915   Enemies unite against a demon horde; Book 1 of...  26.99   
3999                                                         0   
5217                                                         0   

              publisher                 title rank rank_last_week  \
915   Eos/HarperCollins  RIDES A DREAD LEGION   16              0   
3999   

('Susan Mallery',                            _id bestsellers_date published_date  \
4557  5b4aa4ead3089013507dc358       2012-09-28     2012-10-13   
9426  5b4aa4ead3089013507dd65d       2017-07-14     2017-07-29   

                                     amazon_product_url         author  \
4557  http://www.amazon.com/Fools-Gold-Christmas-Sus...  Susan Mallery   
9426  https://www.amazon.com/Secrets-Tulip-Sisters-P...  Susan Mallery   

                                            description price      publisher  \
4557  Both in Fool’s Gold, Calif., only reluctantly,...     0  Harlequin HQN   
9426  Sisters reconnect when one returns to their tu...     0            HQN   

                             title rank rank_last_week weeks_on_list  \
4557       A FOOL'S GOLD CHRISTMAS   18              0             0   
9426  SECRETS OF THE TULIP SISTERS   12              0             1   

          l_author   l_publisher                   l_title  \
4557  susanmallery  harlequinhqn       a

5376                 crucible                 troydenning_crucible  )
('Umberto Eco',                            _id bestsellers_date published_date  \
3629  5b4aa4ead3089013507dbfb8       2011-11-11     2011-11-26   
3657  5b4aa4ead3089013507dbfd4       2011-11-18     2011-12-03   

                                     amazon_product_url       author  \
3629  http://www.amazon.com/The-Prague-Cemetery-Umbe...  Umberto Eco   
3657  http://www.amazon.com/The-Prague-Cemetery-Umbe...  Umberto Eco   

                                            description price  \
3629  A novel based on the twisted history of “The P...    27   
3657  A novel based on the twisted history of “The P...     0   

                      publisher                title rank rank_last_week  \
3629  Houghton Mifflin Harcourt  THE PRAGUE CEMETERY   10              0   
3657  Houghton Mifflin Harcourt  THE PRAGUE CEMETERY   18              0   

     weeks_on_list    l_author              l_publisher            l_titl

In [16]:
print(raw_books['title'].unique())
print(raw_books['l_title'].unique())

['ODD HOURS' 'THE HOST' "LOVE THE ONE YOU'RE WITH" ...
 'ALL WE EVER WANTED' 'SPYMASTER' 'THE LAST TIME I LIED']
['oddhours' 'thehost' 'lovetheoneyourewith' ... 'allweeverwanted'
 'spymaster' 'thelasttimeilied']


In [7]:
#find just one clean lower title in data that has two different real titles

spec_title = raw_books.groupby(['l_title', 'title'])

doubleTitle = ''
for name, group in spec_title:
    if doubleTitle == name[0]:
        print('lower title: %s' % (name[0]))
    doubleTitle = name[0]
    


lower title: crossfire


In [8]:
#find real titles based on single lower case title
filter_titles = raw_books[(raw_books['l_title'] == 'crossfire')].title.unique()
for t in filter_titles:
    print(raw_books[(raw_books['title'] == t)][['author', 'publisher', 'title']].reset_index(drop=True)[:1])

            author      publisher       title
0  James Patterson  Little, Brown  CROSS FIRE
                           author publisher      title
0  Dick Francis and Felix Francis    Putnam  CROSSFIRE


In [ ]:
#so we can conclude that title is clean, there is one case where lc stripped version is the same

In [9]:
print('Total number of authors: %s' % (len(raw_books['author'].unique())))
print('Total number of clean authors: %s' % (len(raw_books['l_author'].unique())))

Total number of authors: 738
Total number of clean authors: 728


In [ ]:
spec_author = raw_books.groupby(['l_author', 'author'])

double_author = ''
for name, group in spec_author:
    if double_author == name[0]:
        print('lower author: %s' % (name[0]))
    double_author = name[0]

In [ ]:
#find real author name based on single lower case author name
filter_auth_col = raw_books[(raw_books['l_author'] == 'baparis')].author.unique()
for t in filter_auth_col:
    print(raw_books[(raw_books['author'] == t)][['author', 'publisher', 'title']].reset_index(drop=True)[:1])

In [ ]:

print('Total number of publisher names: %s' % (len(raw_books['publisher'].unique())))
print('Total number of clean publisher names: %s' % (len(raw_books['l_publisher'].unique())))

In [ ]:
spec_publisher = raw_books.groupby(['l_publisher', 'publisher'])

double_publisher = ''
for name,group in spec_publisher:
    if double_publisher == name[0]:
        print(name[0])
    double_publisher = name[0]


In [ ]:


filter_auth_col = raw_books[(raw_books['l_publisher'] == 'stmartins')].publisher.unique()
for t in filter_auth_col:
    print(raw_books[(raw_books['publisher'] == t)][['publisher']].reset_index(drop=True)[:1])

